In [2]:
import os
import sys
import random
import gzip
import os
import numpy as np

## paths to files. Do not change this
simInputFile = "Q1/word-similarity-dataset"
analogyInputFile = "Q1/word-analogy-dataset"
#analogyInputFile = "Q1/analogy_small.txt"
vectorgzipFile = "Q1/glove.6B.300d.txt.gz"
#vectorTxtFile = "Q1/glove.6B.300d.txt"   # If you extract and use the gz file, use this.
analogyTrainPath = "Q1/wordRep/"
simOutputFile = "Q1/simOutput.csv"
simSummaryFile = "Q1/simSummary.csv"
anaSOln = "Q1/analogySolution.csv"
Q4List = "Q4/wordList.csv"

random.seed(100)

In [3]:
# Similarity Dataset
simDataset = [item.split(" | ") for item in open(simInputFile).read().splitlines()]
# Analogy dataset
analogyDataset = [[stuff.strip() for stuff in item.strip('\n').split('\n')] for item in open(analogyInputFile).read().split('\n\n')]

def vectorExtract(simD = simDataset, anaD = analogyDataset, vect = vectorgzipFile):
    simList = [stuff for item in simD for stuff in item]
    analogyList = [thing for item in anaD for stuff in item[0:4] for thing in stuff.split()]
    simList.extend(analogyList)
    wordList = set(simList)
    print(len(wordList))
    wordDict = dict()
    
    vectorFile = gzip.open(vect, 'r')
    for line in vectorFile:
        if line.split()[0].strip().decode("utf-8") in wordList:
            wordDict[line.split()[0].strip().decode('utf-8')] = line.split()[1:]
    
    
    vectorFile.close()
    print('retrieved', len(wordDict.keys()))
    return wordDict

# Extracting Vectors from Analogy and Similarity Dataset
#validateVectors = vectorExtract()
wordDict = vectorExtract()


5104
retrieved 4790


In [4]:
import random
import os
import sys

path = "./Q1/wordRep/pairs_from_wordnet/"

#Check if all the words exist in word vector
def vectorexists(wordlist, worddict):
    wordlist = wordlist.split('\t')
    for w in wordlist:
        if(w not in worddict):
            return False
    return True

def writeanalogyfile(path, wordDict):
    filelist = os.listdir(path)
    #print(filelist)
    fout = open('analogy_dataset.txt', 'w')
    for file in filelist:
        alist = []
        qfile = open(path+file, 'r')
        qlist = [line.strip() for line in qfile]
        for otherfiles in filelist:
            if(file != otherfiles):
                f = open(path+otherfiles, 'r')
                l = [line.strip() for line in f]
                #print(file, otherfiles, len(l))
                alist.extend(l)
        for q in qlist:
            if(not vectorexists(q, wordDict)):
                continue
            output = [None]*7
            output[0] = q
            option = random.randint(0,4)
            optionpos = random.randint(0, len(qlist)-1)
            while(not vectorexists(qlist[optionpos], wordDict)):
                optionpos = random.randint(0, len(qlist)-1)
            output[option+1] = qlist[optionpos]
            output[6] = chr(ord('a')+option)
            for i in range(1, 6):
                randop = random.randint(0, len(alist)-1)
                while(not vectorexists(alist[randop], wordDict)):
                    randop = random.randint(0, len(alist)-1)
                if(i!=option+1):
                    output[i] = alist[randop]
            for pair in output:
                pair = pair.replace('\t', ' ')
                fout.write(pair+'\n')
            fout.write('\n\n')
            fout.flush()

#writeanalogyfile(path, wordDict)

In [5]:
def get_analogy_input(anaF, wordDict):
    fin = open(anaF, 'r')
    line = fin.readline()
    trainx = []
    trainy = []
    #trainx = np.zeros((1, 3600))
    #trainy = np.zeros((1,1))
    while(line != ""):
        if(line == '\n'):
            line = fin.readline()
            continue
        #x = np.zeros((1, 300))
        #y = np.zeros((1,1))
        x = []
        y = [0]*5
        for i in range(0, 6):
            line = line.strip().split()
            for w in line:
                if w in wordDict:
                    x.extend(wordDict[w])
                    #print(len(wordDict[w]))
            line = fin.readline()
        answer = line.strip()
        answer = ord(answer)-ord('a')
        y[answer] = 1
        #x = np.array(x)
        counter = 0
        if(len(x)<3600):
            line = fin.readline()
            counter += 1
            continue
        trainx.append(x)
        trainy.append(y)
        #trainx = np.vstack((trainx, np.array(x).T))
        #trainy = np.vstack((trainy, np.array(y)))
        line = fin.readline()
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    pos = len(trainx)-1
    return trainx[1:pos], trainx[pos:], trainy[1:pos], trainy[pos:]

In [6]:
trainx, trainy, testx, testy = get_analogy_input(analogyInputFile, wordDict)
if(len(trainx)<5):
    f = open("output/analogySolution.txt", 'w')
    f.write("Too few training examples. Word vectors present for only "+str(len(trainx))+" examples")
    print("Too few training examples. Word vectors present for only ",len(trainx)," examples")
    f.close()
    #exit()
#print(len(trainx), len(trainy))

In [7]:
# Implementation of a simple MLP network with one hidden layer. Tested on the iris data set.
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

import tensorflow as tf
import numpy as np
#from sklearn import datasets
#from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat


def trainnetwork(anaF, wordDict, train_X, test_X, train_y, test_y):
    #train_X, test_X, train_y, test_y = get_iris_data()
    #train_X, test_X, train_y, test_y = get_analogy_input(anaF, wordDict)

    # Layer's sizes
    x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 256                # Number of hidden nodes
    y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(15):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

    train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                             sess.run(predict, feed_dict={X: train_X, y: train_y}))
    test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                             sess.run(predict, feed_dict={X: test_X, y: test_y}))

    print("Iteration = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
          % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()
    return (train_accuracy, test_accuracy)




In [13]:
def crossvalidation(anaF, wordDict):
    train_X, test_X, train_Y, test_Y = get_analogy_input(anaF, wordDict)
    if(len(train_X)<5): 
        return
    train_accuracy = 0
    test_accuracy = 0
    for i in range(5):
        pos1 = int(0.2*i*len(train_X))
        pos2 = int(0.2*(i+1)*len(train_X))
        if(pos2>len(train_X)-1):
            pos2 = len(train_X)
        cvtrainx = np.vstack((train_X[0:pos1], train_X[pos2:len(train_X)]))
        cvtrainy = np.vstack((train_Y[0:pos1], train_Y[pos2:len(train_Y)]))
        cvtestx = train_X[pos1:pos2]
        cvtesty = train_Y[pos1:pos2]
        tr, ts = trainnetwork(anaF, wordDict, cvtrainx, cvtestx, cvtrainy, cvtesty)
        train_accuracy += tr
        test_accuracy += ts
    f = open("output/analogySolution.csv", 'w')
    f.write("After five fold cross validation. \nTrain accuracy: "+str(train_accuracy/5)+". Test accuracy: "+str(test_accuracy/5))

crossvalidation(analogyInputFile, wordDict)

Iteration = 15, train accuracy = 100.00%, test accuracy = 25.00%
Iteration = 15, train accuracy = 100.00%, test accuracy = 41.67%
Iteration = 15, train accuracy = 100.00%, test accuracy = 25.00%
Iteration = 15, train accuracy = 100.00%, test accuracy = 25.00%
Iteration = 15, train accuracy = 100.00%, test accuracy = 30.77%
